In [2]:
pip install miceforest

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 43 kB 592 kB/s eta 0:00:011
     |████████████████████████████████| 3.2 MB 803 kB/s eta 0:00:01     |███████████████████▉            | 2.0 MB 256 kB/s eta 0:00:05
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/eb/ce/fc2c9d1267cc588ebb33a7864da3a6ee08df47a137361cb90ebabb9484ed/lightgbm-3.3.2-py3-none-macosx_10_14_x86_64.macosx_10_15_x86_64.macosx_11_0_x86_64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
# import miceforest as mf
from scipy import stats
from imblearn.over_sampling import SMOTE
from collections import Counter

In [ ]:
mf.__version__

In [4]:
data = pd.read_csv("./output/datalong_en_024.csv")

In [5]:
state_feature_list = [
    'gender',
    'age',
    'age_at_diagnosis',
    'weight',
    'height',
    'ABS',
    'BMI',

    'previous_radiation',
    'previous_chemotherapy',

    'congestive_heart_failure',
    'angina_pectoris',
    'myocardial_infarction',
    'copd',
    'peptic_ulcer',
    'thyroid_disease',
    'diabetes',
    'autoimmune_disease',
    'acquired_immune_deficiency_syndrome',

    'tumor_stage_pM',
    'tumor_stage_pN',
    'tumor_stage_pT',

    'PNI'
]

dynamic_feature_list = [
    'GFR',
    'AST',
    'TBIL',
    'PA',
    'ALP',
    'ALB',
    'ALT',
    'Crea',

    'PLT',
    'Hb',
    'WBC',
    'Lymph',
    'Neut',
    
    'prevention',

    'PS_score'
]

# data imputation

## chronic disease 
慢性病比例较低 处理为diabetes(糖尿病) /peptic_ulcer（消化性溃疡）/ other_chronic（除上述外的其他慢性病）

In [6]:
data["other_chronic"] = data[["congestive_heart_failure", "angina_pectoris", "myocardial_infarction", "copd", "thyroid_disease", 
                              "autoimmune_disease", "acquired_immune_deficiency_syndrome"]].sum(axis = 1)
data["other_chronic"] = data["other_chronic"].map(lambda x: 1 if x >= 1 else 0)

## TNM tumor stage

pM stage：all M0, drop this column 
pN stage：pNx:imputated with pN0; nan: imputed by miceforest  
pT stage：nan: imputed by miceforest   

In [7]:
data["tumor_stage_pN"] = data["tumor_stage_pN"].map(lambda x: "pN0" if x == "pNx" else x)

## ps score
只保留基线

In [8]:
data.rename(columns = {"C1*PS_score": "PS_score"}, inplace = True)

## output variables with missing values

In [22]:
state_feature_list = [
    'gender',
    'age_at_diagnosis',
    'weight',
    'previous_radiation',
    'previous_chemotherapy',
    'peptic_ulcer',
    'diabetes',
    'other_chronic',
    'tumor_stage_pN',
    'tumor_stage_pT',
    'PNI',
    "PS_score"
]

In [10]:
temp_missing = data[["patient_sn", "visit_year", "event", "event#cycle", "cycle_num"] + \
                    state_feature_list +\
                    [name for name in data.columns.to_list() if "C" in name and not "PS_score" in name]]

将数据集整合成填补后一致的结构，不符合患者周期对应的特征取值为np.nan

In [11]:
data_missing = temp_missing.copy()
cycle_list = ['GFR', 'AST', 'TBIL', 'PA', 'ALP', 'ALB', 'ALT', 'Crea', 'PLT', 'Hb', 'WBC', 'Lymph', 'Neut', 'prevention', 'interval']
for i in range(len(data_missing)):
    event_num = data_missing["event#cycle"][i]
    cycle_num = data_missing["cycle_num"][i]
    if np.isnan(event_num):
        num = cycle_num
    else:
        num  = min(event_num, cycle_num)
    num = int(num)
    for j in range(num+1, 9):
        pre = "C" + str(j) + "*"
        for name in cycle_list:
            data_missing[pre+name][i] = np.nan
data_missing

<ipython-input-11-18e1b31ada61>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_missing[pre+name][i] = np.nan
/Users/weiyuna/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,patient_sn,visit_year,event,event#cycle,cycle_num,gender,age,age_at_diagnosis,weight,height,...,C8*ALB,C8*ALT,C8*Crea,C8*PLT,C8*Hb,C8*WBC,C8*Lymph,C8*Neut,C8*prevention,C8*interval
0,002f98cd8201f14bd9620e885a9d7377,2017,1,2.0,8,1,45,45,70.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0142e3b3ee4b56437e9b4904cde760de,2011,0,NaN,8,1,48,48,71.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,45.0
2,0203dc965807312951ecf49ee6fbdbeb,2017,0,NaN,8,1,54,53,71.0,174.0,...,39.3,19.0,63.0,81.0,131.0,4.19,1.67,1.91902,1.0,45.0
3,021657d2866fd95ea6595f938ace2172,2021,0,NaN,6,1,63,63,60.0,178.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,034054cd4706ef79c817079340cdab89,2015,0,NaN,8,1,51,50,59.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,fcd3e4da2f66e7d62bac9337c54a5439,2020,0,NaN,7,1,60,60,56.9,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,fd81c606879ef8ec683f5801faac8b49,2017,1,3.0,8,1,66,65,80.0,175.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,fd9412166385394112a26e951a94df71,2016,0,NaN,7,0,53,53,70.0,162.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,feb4fa9b0e813eb1e2cb836a81a912e3,2017,0,NaN,6,1,57,57,76.0,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# data_missing.to_csv("./output/missing.csv", index  = False)

# multiple imputation

静态指标mice一个，得到填补后的静态数据集后再逐周期填补。

## static variables

In [22]:
# data_missing = pd.read_csv("./output/missing.csv")

In [25]:
state_missing = data_missing[state_feature_list]
state_missing.isnull().sum()

gender                    0
age_at_diagnosis          0
weight                   43
previous_radiation        0
previous_chemotherapy     0
peptic_ulcer              0
diabetes                  0
other_chronic             0
tumor_stage_pN           31
tumor_stage_pT           32
PNI                      85
PS_score                 54
dtype: int64

In [26]:
for col in state_missing.columns:
    print(col + " : " + str(state_missing[col].dtype))

gender : int64
age_at_diagnosis : int64
weight : float64
previous_radiation : int64
previous_chemotherapy : int64
peptic_ulcer : int64
diabetes : int64
other_chronic : int64
tumor_stage_pN : object
tumor_stage_pT : object
PNI : float64
PS_score : float64


In [28]:
for col_name in ['tumor_stage_pN', 'tumor_stage_pT']:
    state_missing[col_name] = state_missing[col_name].astype("category")


<ipython-input-28-46c19a2f3ad8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_missing[col_name] = state_missing[col_name].astype("category")


In [29]:
kernel0 = mf.ImputationKernel(
    state_missing,
    datasets = 1, # number of final completed datasets
    save_all_iterations = True,
    save_models = 1, # whether the model calculating the missing values is saved =0:unsaved; =1:save the final model; =2:save all models
    random_state = 0
)

kernel0.mice(
    iterations = 10,
    # n_jobs = -1
)

print(kernel0)

state_complete = kernel0.complete_data(0) 
state_complete

              Class: ImputationKernel
           Datasets: 1
         Iterations: 10
  Imputed Variables: 5
save_all_iterations: True


,gender,age_at_diagnosis,weight,previous_radiation,previous_chemotherapy,peptic_ulcer,diabetes,other_chronic,tumor_stage_pN,tumor_stage_pT,PNI,PS_score
0,1,45,70.0,0,0,0,0,0,pN0,pT4,40.30870,1.0
1,1,48,71.0,0,0,0,0,0,pN1,pT2,40.31230,1.0
2,1,53,71.0,0,0,0,0,0,pN1,pT2,33.81100,1.0
3,1,63,60.0,0,0,0,0,0,pN0,pT4,37.20990,1.0
4,1,50,59.0,0,0,0,0,0,pN1,pT4,43.61650,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
469,1,60,56.9,0,0,0,0,0,pN2,pT3,35.20815,1.0
470,1,65,80.0,0,0,0,1,0,pN1,pT3,39.30595,1.0
471,0,53,70.0,0,0,1,1,0,pN0,pT1,40.70725,1.0
472,1,57,76.0,0,0,0,0,0,pN1,pT2,42.10765,1.0


In [30]:
# replace the original variables with imputated variables
data_missing_t = data_missing.copy()
for feature in list(state_complete.columns):
    # print(feature)
    data_missing_t[feature] = list(state_complete[feature])
data_missing_t

,patient_sn,visit_year,event,event#cycle,cycle_num,gender,age_at_diagnosis,weight,previous_radiation,previous_chemotherapy,...,C8*ALB,C8*ALT,C8*Crea,C8*PLT,C8*Hb,C8*WBC,C8*Lymph,C8*Neut,C8*prevention,C8*interval
0,002f98cd8201f14bd9620e885a9d7377,2017,1,2.0,8,1,45,70.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0142e3b3ee4b56437e9b4904cde760de,2011,0,NaN,8,1,48,71.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,45.0
2,0203dc965807312951ecf49ee6fbdbeb,2017,0,NaN,8,1,53,71.0,0,0,...,39.3,19.0,63.0,81.0,131.0,4.19,1.67,1.91902,1.0,45.0
3,021657d2866fd95ea6595f938ace2172,2021,0,NaN,6,1,63,60.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,034054cd4706ef79c817079340cdab89,2015,0,NaN,8,1,50,59.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,fcd3e4da2f66e7d62bac9337c54a5439,2020,0,NaN,7,1,60,56.9,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,fd81c606879ef8ec683f5801faac8b49,2017,1,3.0,8,1,65,80.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,fd9412166385394112a26e951a94df71,2016,0,NaN,7,0,53,70.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,feb4fa9b0e813eb1e2cb836a81a912e3,2017,0,NaN,6,1,57,76.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
data_complete0 = data_missing_t[["patient_sn", "visit_year", "event", "event#cycle", "cycle_num"]+ \
                                state_feature_list+ \
                                [name for name in data_missing_t.columns if "prevention" in name]
                               ]

## dynamic features

In [32]:
state_feature_list = [
    'gender',
    'age_at_diagnosis',
    'weight',
    'previous_radiation',
    'previous_chemotherapy',
    'peptic_ulcer',
    'diabetes',
    'other_chronic',
    'tumor_stage_pN',
    'tumor_stage_pT',
    'PNI',
    'PS_score'
]
dynamic_feature_list = ["GFR", "AST",  "TBIL", "PA", "ALP", "ALB", "ALT", "Crea", "PLT", "Hb", "WBC", "Lymph", "Neut"]

In [37]:
def dynamic_mice(num):
    cycle = "C"+str(num)
    cycle_t = data_missing_t[(data_missing_t["cycle_num"] >= num) & \
                             ((data_missing_t["event#cycle"] >= num) | (np.isnan(data_missing_t["event#cycle"])))]
    print(len(cycle_t))
    cycle_missing = cycle_t[state_feature_list + \
                           [cycle+"*"+name for name in dynamic_feature_list]]
    
    for col in cycle_missing.columns:
        if cycle_missing[col].dtype == np.object:
            cycle_missing[col] = cycle_missing[col].astype("category")
    
    kernel = mf.ImputationKernel(
        cycle_missing,
        datasets = 1, # number of final completed datasets
        save_all_iterations = True,
        save_models = 1, # whether the model calculating the missing values is saved =0:unsaved; =1:save the final model; =2:save all models
        random_state = 0
    )
    
    kernel.mice(
        iterations = 15,
        n_jobs = -1
    )
    
    data_complete = kernel.complete_data(0) # which completed datasets is chosen
    data_complete = data_complete[[cycle+"*"+name for name in dynamic_feature_list]]
    
    data_complete["patient_sn"] = list(cycle_t["patient_sn"])
    data_complete[cycle+"*interval"] = list(cycle_t[cycle+"*interval"])
    
    return kernel, data_complete

In [38]:
# data_complete0
kernel1,  data_complete1 = dynamic_mice(1)
data_complete01 = pd.merge(data_complete0, data_complete1, on = "patient_sn", how = "left")

474


<ipython-input-37-19e81191cdf3>:10: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if cycle_missing[col].dtype == np.object:
<ipython-input-37-19e81191cdf3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_missing[col] = cycle_missing[col].astype("category")


In [39]:
kernel2,  data_complete2 = dynamic_mice(2)
data_complete012 = pd.merge(data_complete01, data_complete2, on = "patient_sn", how = "left")

435


<ipython-input-37-19e81191cdf3>:10: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if cycle_missing[col].dtype == np.object:
<ipython-input-37-19e81191cdf3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cycle_missing[col] = cycle_missing[col].astype("category")


In [48]:
kernel3,  data_complete3 = dynamic_mice(3)
data_complete0123 = pd.merge(data_complete012, data_complete3, on = "patient_sn", how = "left")

In [49]:
kernel4,  data_complete4 = dynamic_mice(4)
data_complete01234 = pd.merge(data_complete0123, data_complete4, on = "patient_sn", how = "left")

In [50]:
kernel5,  data_complete5 = dynamic_mice(5)
data_complete012345 = pd.merge(data_complete01234, data_complete5, on = "patient_sn", how = "left")

In [51]:
kernel6,  data_complete6 = dynamic_mice(6)
data_complete0123456 = pd.merge(data_complete012345, data_complete6, on = "patient_sn", how = "left")

In [52]:
kernel7,  data_complete7 = dynamic_mice(7)
data_complete01234567 = pd.merge(data_complete0123456, data_complete7, on = "patient_sn", how = "left")

In [53]:
kernel8,  data_complete8 = dynamic_mice(8)
data_complete012345678 = pd.merge(data_complete01234567, data_complete8, on = "patient_sn", how = "left")

In [54]:
data_complete012345678

,patient_sn,visit_year,event,event#cycle,cycle_num,gender,age_at_diagnosis,weight,previous_radiation,previous_chemotherapy,...,C8*ALP,C8*ALB,C8*ALT,C8*Crea,C8*PLT,C8*Hb,C8*WBC,C8*Lymph,C8*Neut,C8*interval
0,002f98cd8201f14bd9620e885a9d7377,2017,1,2.0,8,1,45,70.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0142e3b3ee4b56437e9b4904cde760de,2011,0,NaN,8,1,48,71.0,0,0,...,104.0,41.7,19.0,55.0,70.0,131.0,3.44,1.32,1.57084,45.0
2,0203dc965807312951ecf49ee6fbdbeb,2017,0,NaN,8,1,53,71.0,0,0,...,87.0,39.3,19.0,63.0,81.0,131.0,4.19,1.67,1.91902,45.0
3,021657d2866fd95ea6595f938ace2172,2021,0,NaN,6,1,63,60.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,034054cd4706ef79c817079340cdab89,2015,0,NaN,8,1,50,59.0,0,0,...,78.0,41.9,26.0,65.0,85.0,126.0,3.57,1.33,1.82484,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,fcd3e4da2f66e7d62bac9337c54a5439,2020,0,NaN,7,1,60,56.9,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,fd81c606879ef8ec683f5801faac8b49,2017,1,3.0,8,1,65,80.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,fd9412166385394112a26e951a94df71,2016,0,NaN,7,0,53,70.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,feb4fa9b0e813eb1e2cb836a81a912e3,2017,0,NaN,6,1,57,76.0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
data_complete012345678.to_csv("./output/complete.csv", index = False)

## after imputation

In [56]:
basic_list = ["GFR", "AST",  "TBIL", "PA", "ALP", "ALB", "ALT", "Crea", "PLT", "Hb", "WBC", "Lymph", "Neut"]
cycle_list = list()
for i in range(1, 9):
    cycle_list = cycle_list + ["C"+str(i)+"*"+name for name in basic_list]

result_list = list()
for column_name in cycle_list:
    x1 = list(data_missing[~data_missing[column_name].isnull()][column_name])
    x2 = list(data_complete012345678[~data_complete012345678[column_name].isnull()][column_name])
    result_list.append(
        {
            "feature": column_name,
            "pvalue": stats.stats.ttest_ind(x1, x2).pvalue
        }
    )

In [57]:
pd.DataFrame(result_list).sort_values(["pvalue"], ascending=True)

,feature,pvalue
16,C2*PA,0.009860
29,C3*PA,0.069483
68,C6*PA,0.257950
37,C3*Lymph,0.365913
35,C3*Hb,0.377823
...,...,...
49,C4*WBC,0.980607
53,C5*AST,0.980936
57,C5*ALB,0.985316
58,C5*ALT,0.986960


# final table (long type)

In [2]:
import pandas as pd
import numpy as np
import miceforest as mf
from scipy import stats

In [3]:
data_complete = pd.read_csv("./output/填补之后的_complete.csv")
data = pd.read_csv("./output/datalong_en_024.csv")
data_prevention = data[["patient_sn"]+[name for name in data.columns if "prevention" in name]]
data_complete = pd.merge(data_complete, data_prevention, on = "patient_sn")

In [18]:
data_complete["id"] = data_complete.index
state_feature = [
    'patient_sn',
    'id',
    'cycle_num',
    'visit_year',
    'gender',
    'age_at_diagnosis',
    'weight',
    'previous_radiation',
    'previous_chemotherapy',
    'peptic_ulcer',
    'diabetes',
    'other_chronic',
    'tumor_stage_pN',
    'tumor_stage_pT',
    'PNI',
    'PS_score'
]

data_state = data_complete[state_feature]

In [24]:
data_complete[["event#cycle","cycle_num"]]

,event#cycle,cycle_num
0,2.0,8
1,NaN,8
2,NaN,8
3,NaN,6
4,NaN,8
...,...,...
469,NaN,7
470,3.0,8
471,NaN,7
472,NaN,6


In [25]:
dynamic_feature_list = ["GFR", "AST",  "TBIL", "PA", "ALP", "ALB", "ALT", "Crea", "PLT", "Hb", "WBC", "Lymph", "Neut", "prevention", "interval"]

dynamic_result = list()
for i in range(len(data_complete)):
    event_cycle = data_complete["event#cycle"][i]
    num_cycle = data_complete["cycle_num"][i]
    if np.isnan(event_cycle):
        cnt = num_cycle
    else:
        cnt = min(event_cycle, num_cycle)
    cnt = int(cnt)
    
    for num in range(1, cnt+1):
        t_dict = dict()
        
        cycle_num = "C"+str(num)+"*"
        t_dict["patient_sn"] = data_complete["patient_sn"][i]
        t_dict["cycle"] = num
        
        for cycle_feature in dynamic_feature_list:
            if cycle_feature == "interval":
                t_dict[cycle_feature] = data_complete["C"+str(num)+"*"+cycle_feature][i]
            elif cycle_feature == "prevention" and num == 1:
                t_dict[cycle_feature] = 0
            else:
                t_dict[cycle_feature] = data_complete["C"+str(num-1)+"*"+cycle_feature][i]
        
        # event    
        if np.isnan(event_cycle):
            t_dict["event"] = 0
        elif num == event_cycle:
            t_dict["event"] = 1
        else:
            t_dict["event"] = 0
            
        # previous cycle interval
        if num == 1:
            t_dict["last_interval"] = 30
        else:
            t_dict["last_interval"] = data_complete["C"+str(num-1)+"*"+"interval"][i]
        
        dynamic_result.append(t_dict)
data_dynamic = pd.DataFrame(dynamic_result)

# the start-time and end-time of present cycle of chemotherapy
data_dynamic["time2"] = data_dynamic.groupby(["patient_sn"])["interval"].cumsum()
data_dynamic["time1"] = data_dynamic["time2"] - data_dynamic["interval"]
data_dynamic.drop(["interval"], axis = 1, inplace = True)
for i in range(len(data_dynamic)):
    if data_dynamic["time1"][i] == data_dynamic["time2"][i]:
        data_dynamic["time2"][i] = data_dynamic["time1"][i]+1

In [27]:
data_long = pd.merge(data_dynamic, data_state, on = "patient_sn", how = "left")
# data_long.to_csv("./output/data_long.csv", index = False)
data_long

,patient_sn,cycle,GFR,AST,TBIL,PA,ALP,ALB,ALT,Crea,...,weight,previous_radiation,previous_chemotherapy,peptic_ulcer,diabetes,other_chronic,tumor_stage_pN,tumor_stage_pT,PNI,PS_score
0,002f98cd8201f14bd9620e885a9d7377,1,114.643724,14.0,18.2,20.10,58.0,40.3,16.0,61.0,...,70.0,0,0,0,0,0,pN0,pT4,40.3087,1.0
1,002f98cd8201f14bd9620e885a9d7377,2,119.627708,56.0,18.4,21.60,62.0,40.4,66.0,55.0,...,70.0,0,0,0,0,0,pN0,pT4,40.3087,1.0
2,0142e3b3ee4b56437e9b4904cde760de,1,98.483763,22.0,13.7,17.70,60.0,40.3,24.0,81.0,...,71.0,0,0,0,0,0,pN1,pT2,40.3123,1.0
3,0142e3b3ee4b56437e9b4904cde760de,2,128.502263,20.0,20.8,20.70,55.0,47.2,21.0,65.0,...,71.0,0,0,0,0,0,pN1,pT2,40.3123,1.0
4,0142e3b3ee4b56437e9b4904cde760de,3,126.152077,37.0,16.5,22.60,51.0,44.2,45.0,66.0,...,71.0,0,0,0,0,0,pN1,pT2,40.3123,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,ffea570e2791a58ecc6032150a84141b,2,103.241483,23.0,9.2,13.31,67.0,35.0,13.0,56.0,...,69.0,0,1,0,0,0,pN0,pT4,35.4088,1.0
2472,ffea570e2791a58ecc6032150a84141b,3,103.154622,15.0,8.5,16.80,54.0,35.2,5.0,56.0,...,69.0,0,1,0,0,0,pN0,pT4,35.4088,1.0
2473,ffea570e2791a58ecc6032150a84141b,4,112.975859,19.0,10.7,16.80,54.0,34.8,6.0,64.0,...,69.0,0,1,0,0,0,pN0,pT4,35.4088,1.0
2474,ffea570e2791a58ecc6032150a84141b,5,102.550959,17.0,9.6,17.60,76.0,39.1,14.0,60.0,...,69.0,0,1,0,0,0,pN0,pT4,35.4088,1.0


# data discretization

In [13]:
data_long = pd.read_csv("./output/data_long.csv")
# most of PS_score is 0, merge values greater than 0 into one categroy
data_long["PS_score_gt_0"] = data_long["PS_score"].map(lambda x: 1 if x == 1 else 0)

data_long["tumor_stage_pN"] = data_long["tumor_stage_pN"].map(lambda x: 0 if x == "pN0" else 1)

In [14]:
continous_feature = [
    "age_at_diagnosis",
    "weight",
    "PNI",
    "GFR",  "TBIL", "ALP", "ALB", "ALT", "Crea", "PLT", "Hb", "WBC",
    "AST", "PA", "Lymph", "Neut"
]

def quartile_feature(feature_name, data):
    """
    将连续变量根据四分位数生成取值1-4的连续变量
    """
    df = data.copy()
    if feature_name == "age_at_diagnosis":
        df[feature_name] = pd.cut(df[feature_name], [0, 60, 120], labels = ["le_60", "lt_60"])
        df["age_at_diagnosis_gt_60"] = df["age_at_diagnosis"].map(lambda x: 1 if x == "lt_60" else 0)
    else:
        quantile_list = df[feature_name].quantile([0, 0.25, 0.5, 0.75, 1]).to_list()
        quantile_list[0] = 0.99*quantile_list[0]
        df["{}_q".format(feature_name)] = pd.cut(df[feature_name], quantile_list, labels = ["1", "2", "3", "4"], right = True)
        df["{}_q".format(feature_name)] = df["{}_q".format(feature_name)].map(lambda x: int(x))
    return df

df = data_long
for feature in continous_feature:
    df = quartile_feature(feature, df)

In [48]:
data_long.to_csv("./output/data_long_quartile_feature.csv", index = False)

In [51]:
df_new = df[[
    'patient_sn',
    'visit_year',
    'cycle_num',
    'cycle',
    'event',

    'gender',
    'tumor_stage_pN',

    'last_interval',
    'prevention',  
    'GFR_q',
    'TBIL_q',
    'ALP_q',
    'ALB_q',
    'ALT_q',
    'Crea_q',
    'PLT_q',
    'Hb_q',
    'WBC_q',
    'AST_q',
    'PA_q',
    'Lymph_q',
    'Neut_q',df_new.to_csv("./output/data_valid.csv", index = False)
    
]]


# Smote

In [6]:
data = pd.read_csv("/Users/weiyuna/Desktop/工作/合作项目/医大一-胃癌/模型/code_202203/output/data_valid.csv")

data_valid = data
i = 8
data_valid = data_valid[data_valid.cycle <= i]

data_train = data_valid[data_valid["visit_year"] <= 2018]
X_train = data_train[[x for x in data.columns if x not in ["patient_sn", "visit_year", "cycle_num", "event"]]]
y_train = data_train["event"]

data_test = data_valid[data_valid["visit_year"] > 2018]
data_test.reset_index(inplace = True)
X_test = data_test[[x for x in data.columns if x not in ["patient_sn", "visit_year", "cycle_num", "event"]]]
y_test = data_test["event"]

In [7]:
smo = SMOTE(random_state = 0)
X_train_smo, y_train_smo = smo.fit_resample(X_train, y_train)
smote_train = X_train_smo
smote_train["event"] = y_train_smo
smote_train["target"] = "train"
print('smote train: {}'.format(Counter(y_train_smo)))

X_test_smo, y_test_smo = smo.fit_resample(X_test, y_test)
smote_test = X_test_smo
smote_test["event"] = y_test_smo
smote_test["target"] = "test"
print('smote test: {}'.format(Counter(y_test_smo)))

smote train: Counter({0: 1935, 1: 1935})
smote test: Counter({0: 395, 1: 395})


In [8]:
smote_data = smote_train.append(smote_test)
# smote_data.to_csv("smote_data.csv", index = False)

/var/folders/09/ybrtr66d0nn9kwdkvqxs98n00000gn/T/ipykernel_76873/2157473135.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  smote_data = smote_train.append(smote_test)
